In [61]:
import pymongo
import pandas as pd
import itertools
import csv
import tqdm

OUTFILE = 'graduates.csv'

In [62]:
client = pymongo.MongoClient()
db = client['uni']

In [69]:
def get_graduates(vpo1_collection):
    return vpo1_collection.aggregate(
        pipeline=[
    {
        '$match': {
            'section': '2.1.3'
        }
    }, {
        '$project': {
            'rows': 0
        }
    }, {
        '$unwind': {
            'path': '$columns', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'columns.name': {
                '$regex': 'Выпуск фактический.*'
            }
        }
    }, {
        '$unwind': {
            'path': '$columns.records', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$project': {
            'region': 1, 
            'funded_by': 1, 
            'time_involvement': 1, 
            # 'section': 1, 
            # 'section_name': 1, 
            'colname': '$columns.name', 
            'group': '$columns.records.group', 
            'program': '$columns.records.name', 
            'value': '$columns.records.value'
        }
    }
])

In [70]:
fieldnames = list(next(get_graduates(db.vpo1)).keys())
fieldnames.remove('_id')
fieldnames.remove('group')
fieldnames

['region', 'funded_by', 'time_involvement', 'colname', 'program', 'value']

In [71]:
with open(OUTFILE, 'w', newline='') as f:
    f.truncate(0)
    writer = csv.DictWriter(f, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()
    for record in tqdm.tqdm(get_graduates(db.vpo1)):
        record['program'] = '{}. {}'.format(record['group'], record['program'])
        for k in list(record.keys()):
            if k not in fieldnames:
                del record[k]
        writer.writerow(record)

56478it [00:03, 18804.28it/s]


In [72]:
df = pd.read_csv(OUTFILE)
df.head()

,region,funded_by,time_involvement,colname,program,value
0,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1. Программы бакалавриата - всего,2232.0
1,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1. Информатика и вычислительная техника,44.0
2,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1. Психология,1.0
3,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1. Экономика,371.0
4,Российская Федерация,Частные,аттестация экстернов,Выпуск фактический с 01.10. прошлого года по 3...,1. Менеджмент,495.0


In [73]:
df[df['program'] == '1. Математика'].head()

,region,funded_by,time_involvement,colname,program,value
419,Российская Федерация,Всего,очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,463.0
933,Российская Федерация,Всего,очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,4.0
2855,Российская Федерация,Всего,очно-заочная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,18.0
3157,Российская Федерация,Всего,очно-заочная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,0.0
3799,Российская Федерация,"Государственные, Муниципальные",очная,Выпуск фактический с 01.10. прошлого года по 3...,1. Математика,463.0


In [74]:
df['colname'].unique()

array(['Выпуск фактический с 01.10. прошлого года по 30.09. текущего года всего (сумма гр. 10, 12, 13, 15)',
       'Выпуск фактический с 01.10. прошлого года по 30.09. текущего года из них лица с ОВЗ, инвалиды, детиинвалиды'],
      dtype=object)